# Coherence Phase Hypothesis — Run All Notebook

**Purpose:** One-click reproducibility notebook (Methods → Experiments → Figures → Video).

- Environment: Conway+Combat competitive cellular automaton (two colors)
- Agents: **Red baseline** (aggressive greedy) vs **Blue coherence agent** (field memory + softmax temperature)
- Experiments: temperature sweep \(T \in \{0.05, 0.15, 0.35, 0.7\}\)
- Outputs: comparative plots + optional MP4 field video

**Date:** 2026-01-28


## 0) Install (Colab only)

If you're running on Google Colab, the next cell installs **ffmpeg** (for MP4 export) and **imageio** (for writing videos). If you don't need video, you can skip it.


In [ ]:
# Colab-only installs (safe to re-run)
try:
    import google.colab  # noqa
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    !apt-get -y install ffmpeg > /dev/null
    !pip -q install imageio
else:
    print("Not in Colab — skipping installs. If you want video, install ffmpeg + imageio locally.")

## 1) Methods: Model + Agents (Reference Implementation)

This section defines:
- Conway+Combat rules
- Coherence fields \(F_{local}, F_g\)
- Red baseline agent
- Blue coherence agent with temperature \(T\)
- Metrics


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass

# -----------------------------
# Parameters
# -----------------------------
@dataclass
class Params:
    H: int = 60
    W: int = 90
    kill_margin: int = 3          # opponent dominance needed to kill a cell
    field_decay: float = 0.985
    local_inject: float = 0.08
    global_inject: float = 0.03
    seed_noise: int = 250         # initial random cells
    steps: int = 500              # timesteps per run

params = Params()

# -----------------------------
# Core dynamics
# -----------------------------
def neighbor_counts(grid: np.ndarray):
    """Return (r, b) neighbor counts for each cell (Moore neighborhood)."""
    red = (grid == 1).astype(np.int32)
    blue = (grid == 2).astype(np.int32)

    def shift_sum(a):
        s = np.zeros_like(a)
        for dy in (-1, 0, 1):
            for dx in (-1, 0, 1):
                if dy == 0 and dx == 0:
                    continue
                s += np.roll(np.roll(a, dy, axis=0), dx, axis=1)
        return s

    r = shift_sum(red)
    b = shift_sum(blue)
    return r, b

def life_step_with_combat(grid: np.ndarray, params: Params, Fg: float, Flocal: np.ndarray):
    """One automaton step: Conway births/survival + color dominance + combat kills."""
    r, b = neighbor_counts(grid)
    n = r + b
    new_grid = np.zeros_like(grid)

    empty = (grid == 0)
    red_cell = (grid == 1)
    blue_cell = (grid == 2)

    # Birth: exactly 3 neighbors; color by majority
    birth = empty & (n == 3)
    red_birth = birth & (r > b)
    blue_birth = birth & (b > r)

    # Tie-break (rare with n==3 but retained for extensibility)
    tie_birth = birth & (r == b)
    if np.any(tie_birth):
        bias = (Fg + Flocal)  # positive favors Red, negative favors Blue
        red_birth |= tie_birth & (bias >= 0)
        blue_birth |= tie_birth & (bias < 0)

    new_grid[red_birth] = 1
    new_grid[blue_birth] = 2

    # Survival: Life window
    survive = (n == 2) | (n == 3)
    new_grid[red_cell & survive] = 1
    new_grid[blue_cell & survive] = 2

    # Combat kill override: if opponent dominates neighborhood strongly
    red_killed = red_cell & (b - r >= params.kill_margin)
    blue_killed = blue_cell & (r - b >= params.kill_margin)
    new_grid[red_killed] = 0
    new_grid[blue_killed] = 0

    return new_grid, r, b

def update_fields(grid: np.ndarray, r: np.ndarray, b: np.ndarray, params: Params, Fg: float, Flocal: np.ndarray):
    """Update coherence fields with decay and coherence injection."""
    Flocal *= params.field_decay
    Fg *= params.field_decay

    coh = np.zeros_like(Flocal, dtype=np.float32)
    red = (grid == 1)
    blue = (grid == 2)

    # Local coherence signal: same-color neighbor advantage
    coh[red] = (r[red] - b[red]).astype(np.float32)
    coh[blue] = (b[blue] - r[blue]).astype(np.float32)
    coh = np.tanh(coh / 3.0)

    Flocal += params.local_inject * coh
    Fg += params.global_inject * float(np.mean(coh))

    return Fg, Flocal

# -----------------------------
# Metrics
# -----------------------------
def boundary_conflict(grid: np.ndarray) -> int:
    """Approximate Red–Blue boundary length via 4-neighborhood adjacency."""
    red = (grid == 1).astype(np.int8)
    blue = (grid == 2).astype(np.int8)

    up_b = np.roll(blue, -1, axis=0); down_b = np.roll(blue, 1, axis=0)
    left_b = np.roll(blue, -1, axis=1); right_b = np.roll(blue, 1, axis=1)

    up_r = np.roll(red, -1, axis=0); down_r = np.roll(red, 1, axis=0)
    left_r = np.roll(red, -1, axis=1); right_r = np.roll(red, 1, axis=1)

    rb = (red & (up_b | down_b | left_b | right_b)).sum()
    br = (blue & (up_r | down_r | left_r | right_r)).sum()

    return int((rb + br) / 2)

# -----------------------------
# Agent scoring components
# -----------------------------
def dominance_features(grid: np.ndarray, color: int):
    """Local features used by both agents to pursue dominance, filling, and kill pressure."""
    r, b = neighbor_counts(grid)
    n = r + b
    empty = (grid == 0)

    if color == 1:
        same, opp = r, b
    else:
        same, opp = b, r

    # Proxy for survival potential if a cell is placed here
    survive_potential = (same >= 2).astype(np.int32)

    # Proxy for creating births (near empty cells with n==2)
    birth_potential = ((empty) & (n == 2) & (same >= opp)).astype(np.int32)

    # Proxy for kill pressure / contest intensity
    kill_pressure = opp.astype(np.int32)

    # Prefer space (lower neighbor density) for expansion
    space = (8 - n).astype(np.int32)

    return same, opp, survive_potential, birth_potential, kill_pressure, space

# -----------------------------
# Agents
# -----------------------------
def red_baseline_agent(grid: np.ndarray):
    """Baseline Red agent: aggressive greedy dominance (short-horizon optimizer)."""
    empties = np.argwhere(grid == 0)
    if len(empties) == 0:
        return None

    same, opp, survive_p, birth_p, kill_p, space = dominance_features(grid, color=1)

    # Strongly favor immediate dominance and opponent pressure
    score = (
        2.2 * same
        - 1.0 * opp
        + 2.0 * survive_p
        + 1.6 * birth_p
        + 0.8 * kill_p
        + 0.2 * space
    )

    vals = score[empties[:, 0], empties[:, 1]]
    idx = int(np.argmax(vals))
    y, x = empties[idx]
    return int(y), int(x)

def blue_coherence_agent(grid: np.ndarray, Fg: float, Flocal: np.ndarray, temperature: float = 0.35):
    """Blue agent: long-horizon coherence + explicit dominance/kill incentives.

    Chooses an empty cell via softmax sampling over a score function.
    Temperature T controls determinism/exploration:
      - low T -> near-greedy (deterministic)
      - high T -> exploratory (stochastic)
    """
    empties = np.argwhere(grid == 0)
    if len(empties) == 0:
        return None

    same, opp, survive_p, birth_p, kill_p, space = dominance_features(grid, color=2)

    # Field bias (sign flipped so positive favors Blue)
    field_bias = -(Fg + Flocal)
    conflict = opp

    score = (
        1.2 * same
        - 1.4 * conflict
        + 0.9 * field_bias
        + 1.4 * survive_p
        + 1.2 * birth_p
        + 0.9 * kill_p
        + 0.2 * space
    )

    vals = score[empties[:, 0], empties[:, 1]].astype(np.float64)
    vals -= vals.max()  # numerical stability

    T = max(1e-6, float(temperature))
    probs = np.exp(vals / T)
    probs /= probs.sum()

    pick = int(np.random.choice(len(empties), p=probs))
    y, x = empties[pick]
    return int(y), int(x)


## 2) Experiments: Single-run + Temperature Sweep

This section runs:
1) A single run at default temperature (for sanity check)
2) A sweep over \(T \in \{0.05, 0.15, 0.35, 0.7\}\)

Each run returns histories for:
- Red population
- Blue population
- Boundary conflict
- Global field \(F_g\)


In [ ]:
def run_simulation(temp: float, seed: int = 42, params: Params = params):
    """Run one episode for a given Blue temperature; return metric histories."""
    rng = np.random.default_rng(seed)

    grid = np.zeros((params.H, params.W), dtype=np.int8)
    Flocal = np.zeros((params.H, params.W), dtype=np.float32)
    Fg = 0.0

    # Initial random seed state
    for _ in range(params.seed_noise):
        y = int(rng.integers(0, params.H))
        x = int(rng.integers(0, params.W))
        grid[y, x] = int(rng.choice([1, 2]))

    hist = {"red": [], "blue": [], "conflict": [], "Fg": []}

    for _ in range(params.steps):
        # Red places one cell
        m = red_baseline_agent(grid)
        if m is not None:
            y, x = m
            if grid[y, x] == 0:
                grid[y, x] = 1

        # Blue places one cell
        m = blue_coherence_agent(grid, Fg, Flocal, temperature=temp)
        if m is not None:
            y, x = m
            if grid[y, x] == 0:
                grid[y, x] = 2

        # Automaton update + field update
        grid2, r, b = life_step_with_combat(grid, params, Fg, Flocal)
        Fg, Flocal = update_fields(grid2, r, b, params, Fg, Flocal)
        grid = grid2

        # Metrics
        hist["red"].append(int((grid == 1).sum()))
        hist["blue"].append(int((grid == 2).sum()))
        hist["conflict"].append(boundary_conflict(grid))
        hist["Fg"].append(float(Fg))

    return hist

# --- Sanity check run ---
default_T = 0.35
hist0 = run_simulation(default_T, seed=42)
print("Sanity run done. Final counts:",
      "Red =", hist0["red"][-1],
      "Blue =", hist0["blue"][-1],
      "Fg =", round(hist0["Fg"][-1], 4),
      "Conflict =", hist0["conflict"][-1])

plt.figure(figsize=(12,4))
plt.plot(hist0["red"], label="Red")
plt.plot(hist0["blue"], label="Blue")
plt.title(f"Sanity run populations (T={default_T})")
plt.xlabel("timestep"); plt.ylabel("cells")
plt.legend(); plt.grid(True); plt.show()

# --- Sweep ---
temps = [0.05, 0.15, 0.35, 0.7]
seed = 42
results = {t: run_simulation(t, seed=seed) for t in temps}
print("Sweep complete:", list(results.keys()))


## 3) Figures: Comparative Plots

Each figure overlays the four temperatures on the same axis.


In [ ]:
# Blue population
plt.figure(figsize=(12,5))
for t in temps:
    plt.plot(results[t]["blue"], label=f"Blue (T={t})")
plt.title("Blue population vs temperature")
plt.xlabel("timestep"); plt.ylabel("blue cells")
plt.legend(); plt.grid(True); plt.show()

# Red population
plt.figure(figsize=(12,5))
for t in temps:
    plt.plot(results[t]["red"], label=f"Red (T={t})")
plt.title("Red population vs temperature (same opponent across runs)")
plt.xlabel("timestep"); plt.ylabel("red cells")
plt.legend(); plt.grid(True); plt.show()

# Global field
plt.figure(figsize=(12,5))
for t in temps:
    plt.plot(results[t]["Fg"], label=f"Fg (T={t})")
plt.title("Global field Fg vs temperature")
plt.xlabel("timestep"); plt.ylabel("Fg")
plt.legend(); plt.grid(True); plt.show()

# Boundary conflict
plt.figure(figsize=(12,5))
for t in temps:
    plt.plot(results[t]["conflict"], label=f"Conflict (T={t})")
plt.title("Boundary conflict vs temperature")
plt.xlabel("timestep"); plt.ylabel("conflict edges")
plt.legend(); plt.grid(True); plt.show()


## 4) Video: Render the Actual Field (Optional)

This section renders an MP4 showing the grid evolution for a chosen temperature.

- In Colab: make sure you ran the install cell.
- Locally: install `imageio` and ensure `ffmpeg` is available.


In [ ]:
# Choose a temperature for video
chosen_T = 0.35
video_steps = 300
fps = 30
out_path = f"life_ai_vs_ai_T{chosen_T}.mp4"

def render_video(temp: float = 0.35, seed: int = 42, steps: int = 300, fps: int = 30,
                 out_path: str = "life_ai_vs_ai.mp4", params: Params = params):
    """Render a video of the evolving grid. Requires imageio + ffmpeg."""
    import imageio.v2 as imageio

    rng = np.random.default_rng(seed)

    grid = np.zeros((params.H, params.W), dtype=np.int8)
    Flocal = np.zeros((params.H, params.W), dtype=np.float32)
    Fg = 0.0

    for _ in range(params.seed_noise):
        y = int(rng.integers(0, params.H))
        x = int(rng.integers(0, params.W))
        grid[y, x] = int(rng.choice([1, 2]))

    writer = imageio.get_writer(out_path, fps=fps)

    for t in range(steps):
        # Red move
        m = red_baseline_agent(grid)
        if m is not None:
            y, x = m
            if grid[y, x] == 0:
                grid[y, x] = 1

        # Blue move
        m = blue_coherence_agent(grid, Fg, Flocal, temperature=temp)
        if m is not None:
            y, x = m
            if grid[y, x] == 0:
                grid[y, x] = 2

        # Step
        grid2, r, b = life_step_with_combat(grid, params, Fg, Flocal)
        Fg, Flocal = update_fields(grid2, r, b, params, Fg, Flocal)
        grid = grid2

        # Render frame
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.imshow(grid, vmin=0, vmax=2, interpolation="nearest")
        ax.set_title(
            f"Conway+Combat | t={t} | T={temp} | Fg={Fg:+.3f} | R={(grid==1).sum()} B={(grid==2).sum()}"
        )
        ax.axis("off")

        fig.canvas.draw()
        frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
        frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        writer.append_data(frame)
        plt.close(fig)

    writer.close()
    return out_path

# Render + display
try:
    path = render_video(temp=chosen_T, seed=42, steps=video_steps, fps=fps, out_path=out_path)
    print("Saved:", path)

    from IPython.display import Video, display
    display(Video(path, embed=True))
except Exception as e:
    print("Video render failed:", e)
    print("If you're on Colab, run the install cell. If local, install imageio and ffmpeg.")
